In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [2]:
def load_data_from_folder(folder, label):
    data = []
    
    for file in os.listdir(folder):  # Iterate over subject folders
        for subfile in os.listdir(os.path.join(folder, file)):  # Iterate over movement trials
            # Store all six CSVs for this movement
            motion_data = []
            
            # Sort files but only include CSVs
            csv_files = sorted([f for f in os.listdir(os.path.join(folder, file, subfile)) if f.endswith(".csv")])
            
            # Skip if there are not exactly 6 CSV files (prevent stacking errors)
            if len(csv_files) != 6:
                print(f"Skipping {subfile}: Expected 6 CSVs, found {len(csv_files)}")
                continue
            
            # Read each CSV and append its transposed values
            for subsubfile in csv_files:
                full_path = os.path.join(folder, file, subfile, subsubfile)
                df = pd.read_csv(full_path, header=None)
                motion_data.append(df.values.T)  # Transpose to (251, 50)

            # Ensure all 6 sensor readings are stacked properly
            if len(motion_data) == 6:
                motion_array = np.stack(motion_data, axis=0).transpose(2, 0, 1)  # Shape: (50, 6, 251)
                data.append(motion_array)
            else:
                print(f"Skipping {subfile}: Incomplete data")

    return np.array(data), np.full(len(data), label)  # Return data & labels

In [ ]:
from tensorflow.keras.utils import to_categorical

thumbs_up_path = "/Users/eric3/LocalDocuments/treehacks25/gesture_classifier/HGAG-DATA/HGAG-DATA1/Thumb Up"
wrist_ext_path = "/Users/eric3/LocalDocuments/treehacks25/gesture_classifier/HGAG-DATA/HGAG-DATA1/Wrist Extension"
flick_ext_path = "/Users/eric3/LocalDocuments/treehacks25/gesture_classifier/HGAG-DATA/HGAG-DATA1/Index Finger Flick"

# Load data for both gestures
thumbs_up_data, thumbs_up_labels = load_data_from_folder(thumbs_up_path, 0)
wrist_ext_data, wrist_ext_labels = load_data_from_folder(wrist_ext_path, 1)
flick_ext_data, flick_ext_labels = load_data_from_folder(flick_ext_path, 2)

# Combine datasets
X = np.concatenate((thumbs_up_data, wrist_ext_data, flick_ext_data), axis=0)  # (samples, 50, 6, 251)
y = np.concatenate((thumbs_up_labels, wrist_ext_labels, flick_ext_labels), axis=0)
y = to_categorical(y, num_classes=3)  # One-hot encode labels (shape will be (num_samples, 3))

X = X.reshape(-1, 6, 250)  # (86  50 = 4300, 6, 250)
y = np.repeat(y, 50, axis=0)  # Now y will have shape (6450, 3)

# Check shape
print("X shape:", X.shape)  # Expected: (num_samples, 50, 6, 251)
print("y shape:", y.shape)  # Expected: (num_samples,)

Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0
Skipping .mat: Expected 6 CSVs, found 0


In [ ]:
import numpy as np

# Compute min and max for each of the 6 columns (dimension 1)
min_per_column = np.mean(X, axis=(0, 2))
max_per_column = np.mean(X, axis=(0, 2))

print("Mean values for each of the 6 columns:", min_per_column)
print("Maximum values for each of the 6 columns:", max_per_column)

Minimum values for each of the 6 columns: [ 5.61069281e-03 -3.75340097e-03 -1.50271908e-02  2.65557548e-05
 -3.89146866e-05 -1.04058786e-05]
Maximum values for each of the 6 columns: [ 5.61069281e-03 -3.75340097e-03 -1.50271908e-02  2.65557548e-05
 -3.89146866e-05 -1.04058786e-05]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_train = X_train.reshape((X_train.shape[0], 250, 6))  # (3440, 250, 6)
X_test = X_test.reshape((X_test.shape[0], 250, 6))
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5160, 250, 6) (1290, 250, 6) (5160, 3) (1290, 3)


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten

# Define CNN + LSTM Model for Multiclass Classification
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(250, 6)),
    MaxPooling1D(pool_size=2, padding='same'),
    Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2, padding='same'),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')  # 3-class output
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=13, batch_size=16)

Epoch 1/13
323/323 [==============================] - 10s 27ms/step - loss: 0.7151 - accuracy: 0.6591 - val_loss: 0.2312 - val_accuracy: 0.9248
Epoch 2/13
323/323 [==============================] - 8s 25ms/step - loss: 0.2135 - accuracy: 0.9287 - val_loss: 0.1904 - val_accuracy: 0.9256
Epoch 3/13
323/323 [==============================] - 8s 25ms/step - loss: 0.1354 - accuracy: 0.9537 - val_loss: 0.1300 - val_accuracy: 0.9465
Epoch 4/13
323/323 [==============================] - 8s 26ms/step - loss: 0.1215 - accuracy: 0.9605 - val_loss: 0.0900 - val_accuracy: 0.9581
Epoch 5/13
323/323 [==============================] - 8s 25ms/step - loss: 0.0806 - accuracy: 0.9777 - val_loss: 0.0534 - val_accuracy: 0.9853
Epoch 6/13
323/323 [==============================] - 8s 26ms/step - loss: 0.0723 - accuracy: 0.9806 - val_loss: 0.0827 - val_accuracy: 0.9752
Epoch 7/13
323/323 [==============================] - 8s 25ms/step - loss: 0.0640 - accuracy: 0.9798 - val_loss: 0.0447 - val_accuracy: 0.987

In [10]:
# save the model
model.save('gesture_classifier_3classes.keras')

In [11]:
from tensorflow.keras.models import load_model
import tensorflow as tf

# Clear the session to reset the state
tf.keras.backend.clear_session()

# Reload the model
model = load_model('gesture_classifier.keras')

AttributeError: 'Adam' object has no attribute 'build'

In [11]:
import coremltools as ct

# Assuming 'model' is your trained model
mlmodel = ct.convert(model, source="tensorflow", minimum_deployment_target=ct.target.watchOS10)

# Save the converted model
mlmodel.save("gesture_classifier_3classes.mlpackage")

TensorFlow version 2.13.0 has not been tested with coremltools. You may run into unexpected errors. TensorFlow 2.12.0 is the most recent version that has been tested.
Converting TF Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 99189.62 ops/s]
Input ls elem type unknown. Override with <class 'coremltools.converters.mil.mil.types.type_tensor.tensor.<locals>.tensor'>
Converting TF Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 107743.59 ops/s]
Input ls elem type unknown. Override with <class 'coremltools.converters.mil.mil.types.type_tensor.tensor.<locals>.tensor'>
Converting TF Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 18034.20 ops/s]
Saving value type of int64 into a builtin type of int32, might lose precision!
Saving value type of int64 into a builtin type of int32, might lose precision!
Converting TF Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 67963.26 ops/s]
Input ls elem type unknown. Override with <class 'coremltools.converte